In [ ]:
!git clone https://github.com/ankanbhunia/Handwriting-Transformers
%cd Handwriting-Transformers
!pip install --upgrade --no-cache-dir gdown
!gdown --id 16g9zgysQnWk7-353_tMig92KsZsrcM6k && unzip files.zip && rm files.zip

In [38]:
import os
import time
from data.dataset import TextDataset, TextDatasetval
import torch
import cv2
import os
import numpy as np
from models.model import TRGAN
from params import *
from torch import nn
from data.dataset import get_transform
import pickle
from PIL import Image
import tqdm
import shutil

In [5]:
!pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)


In [41]:
text = "মেশিন লার্নিং হলো এমন একটি প্রযুক্তি যেখানে কম্পিউটার ডেটা থেকে শিখে নিজেই সিদ্ধান্ত নিতে পারে, সরাসরি প্রোগ্রাম না লিখেই।"
output_path = 'results'

In [ ]:
# model_path = './saved_models/Final_Bangla/model.pth'
 data_path = 'BN-EN-IAM-MULTI.pickle' #(iam)
#model_path = 'files/cvl_model.pth'; data_path = 'files/CVL-32.pickle' #(cvl)
model_path = 'files/iam_model.pth'; data_path = 'files/CVL-32.pickle' #(iam-cvl-cross)
#model_path = 'files/cvl_model.pth'; data_path = 'files/IAM-32.pickle' #(cvl-iam-cross)#

In [4]:
!pip install controlnet_aux


   ---------------------------------------- 0.0/561.5 kB ? eta -:--:--
   ---------------------------------------- 561.5/561.5 kB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   -- ------------------------------------- 2.1/38.9 MB 29.6 MB/s eta 0:00:02
   -- ------------------------------------- 2.6/38.9 MB 6.0 MB/s eta 0:00:07
   --- ------------------------------------ 3.7/38.9 MB 5.7 MB/s eta 0:00:07
   ------ --------------------------------- 6.6/38.9 MB 7.7 MB/s eta 0:00:05
   ------- -------------------------------- 6.8/38.9 MB 7.9 MB/s eta 0:00:05
   ------- -------------------------------- 6.8/38.9 MB 7.9 MB/s eta 0:00:05
   ------- -------------------------------- 7.3/38.9 MB 5.0 MB/s eta 0:00:07
   -------- ------------------------------- 7.9/38.9 MB 4.7 MB/s eta 0:00:07
   --------- ------------------------------ 9.2/38.9 MB 5.0 MB/s eta 0:00:06
   ----------- ---------------------------- 11.3/38.9 MB 5.7 MB/s eta 0:00:05
   ----

In [5]:
import torch
print(torch.version.cuda)      # Shows CUDA version PyTorch was built with
print(torch.backends.cudnn.version())  # cuDNN version (if available)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA")


12.1
90100
1
NVIDIA GeForce RTX 4090


In [42]:
print ('(1) Loading dataset files...')

TextDatasetObjval = TextDatasetval(base_path = data_path, num_examples = 15)
datasetval = torch.utils.data.DataLoader(
            TextDatasetObjval,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObjval.collate_fn)

print ('(2) Loading model...')

model = TRGAN()
model.load_state_dict(torch.load(model_path))
print (model_path+' : Model loaded Successfully')

print ('(3) Loading text content...')
text_encode =  [j.encode() for j in text.split(' ')]
eval_text_encode, eval_len_text = model.netconverter.encode(text_encode)
eval_text_encode = eval_text_encode.to('cuda:0').repeat(batch_size, 1, 1)


(1) Loading dataset files...
(2) Loading model...
initialize network with N02
initialize network with N02
initialize network with N02


C:\Users\DELL\AppData\Local\Temp\ipykernel_5304\2809497860.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


./saved_models/Final_Bangla/model.pth : Model loaded Successfully
(3) Loading text content...


In [11]:
!gdown 1kRWVk8cGCR_YIcQF-8o0NLub-DDoHuan

Downloading...
From (original): https://drive.google.com/uc?id=1kRWVk8cGCR_YIcQF-8o0NLub-DDoHuan
From (redirected): https://drive.google.com/uc?id=1kRWVk8cGCR_YIcQF-8o0NLub-DDoHuan&confirm=t&uuid=5b6642ee-7ea9-47ba-9543-084877c87ec6
To: d:\New folder\Handwriting-Transformers\model450_pretrained_BN.pth

  0%|          | 0.00/566M [00:00<?, ?B/s]
  0%|          | 524k/566M [00:00<04:14, 2.22MB/s]
  0%|          | 1.57M/566M [00:00<01:59, 4.72MB/s]
  1%|          | 3.67M/566M [00:00<01:20, 6.96MB/s]
  1%|          | 5.24M/566M [00:00<01:02, 9.02MB/s]
  1%|          | 6.82M/566M [00:00<00:52, 10.5MB/s]
  1%|▏         | 8.39M/566M [00:00<00:51, 10.7MB/s]
  2%|▏         | 9.96M/566M [00:01<00:50, 10.9MB/s]
  2%|▏         | 11.5M/566M [00:01<00:50, 10.9MB/s]
  2%|▏         | 13.1M/566M [00:01<00:50, 11.0MB/s]
  3%|▎         | 14.7M/566M [00:01<00:49, 11.1MB/s]
  3%|▎         | 16.3M/566M [00:01<00:50, 10.9MB/s]
  3%|▎         | 17.8M/566M [00:01<00:50, 10.8MB/s]
  3%|▎         | 19.4M/566M [0

In [43]:

if os.path.isdir(output_path): shutil.rmtree(output_path)
os.makedirs(output_path, exist_ok = True)

for i,data_val in enumerate(tqdm.tqdm(datasetval)): 

    page_val = model._generate_page(data_val['simg'].to(DEVICE), data_val['swids'], eval_text_encode,eval_len_text)

    cv2.imwrite(output_path+'/image' + str(i) + '.png', page_val*255)

print ('\nOutput images saved in : ' + output_path)

100%|██████████| 11/11 [00:02<00:00,  5.13it/s]


Output images saved in : results


In [2]:
!pip install wandb

   ---------------------------------------- 0.0/21.6 MB ? eta -:--:--
   -- ------------------------------------- 1.6/21.6 MB 12.0 MB/s eta 0:00:02
   ------- -------------------------------- 4.2/21.6 MB 12.6 MB/s eta 0:00:02
   ------------ --------------------------- 6.6/21.6 MB 11.8 MB/s eta 0:00:02
   ------------- -------------------------- 7.3/21.6 MB 12.2 MB/s eta 0:00:02
   ---------------- ----------------------- 8.9/21.6 MB 9.5 MB/s eta 0:00:02
   ------------------- -------------------- 10.5/21.6 MB 9.0 MB/s eta 0:00:02
   ---------------------- ----------------- 12.1/21.6 MB 9.0 MB/s eta 0:00:02
   ------------------------ --------------- 13.4/21.6 MB 8.7 MB/s eta 0:00:01
   --------------------------- ------------ 14.9/21.6 MB 8.5 MB/s eta 0:00:01
   ------------------------------- -------- 16.8/21.6 MB 8.5 MB/s eta 0:00:01
   ---------------------------------- ----- 18.6/21.6 MB 8.5 MB/s eta 0:00:01
   ------------------------------------- -- 20.4/21.6 MB 8.6 MB/s eta 0:0

In [16]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["WANDB_API_KEY"] = ""

from pathlib import Path
import time
from data.dataset import TextDataset, TextDatasetval
from models import create_model
import torch
import cv2
import os
import numpy as np
from itertools import cycle
from scipy import linalg
from models.model import TRGAN
from params import *
from torch import nn
import wandb

KeyboardInterrupt: 

In [32]:
wandb.init(project="bangla-style", name = EXP_NAME)

init_project()

TextDatasetObj = TextDataset(num_examples = NUM_EXAMPLES)
dataset = torch.utils.data.DataLoader(
            TextDatasetObj,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObj.collate_fn)

TextDatasetObjval = TextDatasetval(num_examples = NUM_EXAMPLES)
datasetval = torch.utils.data.DataLoader(
            TextDatasetObjval,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObjval.collate_fn)

model = TRGAN()

os.makedirs('saved_models', exist_ok = True)
MODEL_PATH = os.path.join('saved_models', EXP_NAME)
if os.path.isdir(MODEL_PATH) and RESUME: 
    model.load_state_dict(torch.load(MODEL_PATH+'/model.pth'))
    print (MODEL_PATH+' : Model loaded Successfully')
else: 
    if not os.path.isdir(MODEL_PATH): os.mkdir(MODEL_PATH)


for epoch in range(EPOCHS):    

    
    start_time = time.time()
    
    for i,data in enumerate(dataset): 

        if (i % NUM_CRITIC_GOCR_TRAIN) == 0:

            model._set_input(data)
            model.optimize_G_only()
            model.optimize_G_step()

        if (i % NUM_CRITIC_DOCR_TRAIN) == 0:

            model._set_input(data)
            model.optimize_D_OCR()
            model.optimize_D_OCR_step()

        if (i % NUM_CRITIC_GWL_TRAIN) == 0:

            model._set_input(data)
            model.optimize_G_WL()
            model.optimize_G_step()

        if (i % NUM_CRITIC_DWL_TRAIN) == 0:

            model._set_input(data)
            model.optimize_D_WL()
            model.optimize_D_WL_step()

    end_time = time.time()
    data_val = next(iter(datasetval))
    losses = model.get_current_losses()
    page = model._generate_page(model.sdata, model.input['swids'])
    page_val = model._generate_page(data_val['simg'].to(DEVICE), data_val['swids'])

    
    wandb.log({'loss-G': losses['G'],
                'loss-D': losses['D'], 
                'loss-Dfake': losses['Dfake'],
                'loss-Dreal': losses['Dreal'],
                'loss-OCR_fake': losses['OCR_fake'],
                'loss-OCR_real': losses['OCR_real'],
                'loss-w_fake': losses['w_fake'],
                'loss-w_real': losses['w_real'],
                'epoch' : epoch,
                'timeperepoch': end_time-start_time,
                
                })

                
    
    wandb.log({ "result":[wandb.Image(page, caption="page"),wandb.Image(page_val, caption="page_val")],
                })

    

    print ({'EPOCH':epoch, 'TIME':end_time-start_time, 'LOSSES': losses})

    if epoch % SAVE_MODEL == 0: torch.save(model.state_dict(), MODEL_PATH+ '/model.pth')
    if epoch % SAVE_MODEL_HISTORY == 0: torch.save(model.state_dict(), MODEL_PATH+ '/model'+str(epoch)+'.pth')

NameError: name 'wandb' is not defined

In [ ]:
import wandb
from comet_ml import start
from comet_ml.integration.pytorch import log_model
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["WANDB_API_KEY"] = ""

from pathlib import Path
import time
from data.dataset import TextDataset, TextDatasetval
from models import create_model
import torch
import cv2
import os
import numpy as np
from itertools import cycle
from scipy import linalg
from models.model import TRGAN
from params import *
from torch import nn


experiment = start(
  api_key="6xk1Nmcm6P2OmkiUlYSqe4IqV",
  project_name="bangla-handwriting",
  workspace="rabib-jahin"
)
def main():

    # wandb.init(project="bangla-final", name = EXP_NAME)

    init_project()
    model_path = 'model450_pretrained_BN.pth'

    TextDatasetObj = TextDataset(num_examples = NUM_EXAMPLES)
    dataset = torch.utils.data.DataLoader(
                TextDatasetObj,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0,
                
                pin_memory=True, drop_last=True,
                collate_fn=TextDatasetObj.collate_fn)

    TextDatasetObjval = TextDatasetval(num_examples = NUM_EXAMPLES)
    datasetval = torch.utils.data.DataLoader(
                TextDatasetObjval,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0,
                pin_memory=True, drop_last=True,
                collate_fn=TextDatasetObjval.collate_fn)

    model = TRGAN()
    # model.load_state_dict(torch.load(model_path))

    os.makedirs('saved_models', exist_ok = True)
    MODEL_PATH = os.path.join('saved_models', EXP_NAME)
    if os.path.isdir(MODEL_PATH) and RESUME: 
        model.load_state_dict(torch.load(MODEL_PATH+'/model.pth'))
        print (MODEL_PATH+' : Model loaded Successfully')
    else: 
        if not os.path.isdir(MODEL_PATH): os.mkdir(MODEL_PATH)


    for epoch in range(EPOCHS):    

        
        start_time = time.time()
        
        for i,data in enumerate(dataset): 

            if (i % NUM_CRITIC_GOCR_TRAIN) == 0:

                model._set_input(data)
                model.optimize_G_only()
                model.optimize_G_step()

            if (i % NUM_CRITIC_DOCR_TRAIN) == 0:

                model._set_input(data)
                model.optimize_D_OCR()
                model.optimize_D_OCR_step()

            if (i % NUM_CRITIC_GWL_TRAIN) == 0:

                model._set_input(data)
                model.optimize_G_WL()
                model.optimize_G_step()

            if (i % NUM_CRITIC_DWL_TRAIN) == 0:

                model._set_input(data)
                model.optimize_D_WL()
                model.optimize_D_WL_step()

        end_time = time.time()
        data_val = next(iter(datasetval))
        losses = model.get_current_losses()
        page = model._generate_page(model.sdata, model.input['swids'])
        page_val = model._generate_page(data_val['simg'].to(DEVICE), data_val['swids'])
        experiment.log_metric("loss-G", losses['G'], step=epoch)
        experiment.log_metric("loss-D", losses['D'], step=epoch)
        experiment.log_metric("loss-Dfake", losses['Dfake'], step=epoch)
        experiment.log_metric("loss-Dreal", losses['Dreal'], step=epoch)
        experiment.log_metric("loss-OCR_fake", losses['OCR_fake'], step=epoch)
        experiment.log_metric("loss-OCR_real", losses['OCR_real'], step=epoch)
        experiment.log_metric("loss-w_fake", losses['w_fake'], step=epoch)
        experiment.log_metric("loss-w_real", losses['w_real'], step=epoch)
        experiment.log_metric("epoch", epoch, step=epoch)
        experiment.log_metric("timeperepoch", end_time - start_time, step=epoch)

  
        
        # wandb.log({'loss-G': losses['G'],
        #             'loss-D': losses['D'], 
        #             'loss-Dfake': losses['Dfake'],
        #             'loss-Dreal': losses['Dreal'],
        #             'loss-OCR_fake': losses['OCR_fake'],
        #             'loss-OCR_real': losses['OCR_real'],
        #             'loss-w_fake': losses['w_fake'],
        #             'loss-w_real': losses['w_real'],
        #             'epoch' : epoch,
        #             'timeperepoch': end_time-start_time,
                    
        #             })

                    
        
        # wandb.log({ "result":[wandb.Image(page, caption="page"),wandb.Image(page_val, caption="page_val")],
        #             })
        experiment.log_image(page, name="page")
        experiment.log_image(page_val, name="page_val")

        

        print ({'EPOCH':epoch, 'TIME':end_time-start_time, 'LOSSES': losses})

        if epoch % SAVE_MODEL == 0: torch.save(model.state_dict(), MODEL_PATH+ '/model.pth')
        if epoch % SAVE_MODEL_HISTORY == 0: torch.save(model.state_dict(), MODEL_PATH+ '/model'+str(epoch)+'.pth')


if __name__ == "__main__":
    
    main()


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/rabib-jahin/bangla-handwriting/4114673cfec348c6ac20d16be1158488

d:\deep_learning\software\miniconda3\envs\ndssxfed\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\deep_learning\software\miniconda3\envs\ndssxfed\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `

initialize network with N02
initialize network with N02
initialize network with N02


d:\deep_learning\software\miniconda3\envs\ndssxfed\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
d:\deep_learning\software\miniconda3\envs\ndssxfed\Lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
C:\Users\DELL\AppData\Local\Temp\ipykernel_13704\1299828820.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpi

saved_models\Final_Bangla : Model loaded Successfully
{'EPOCH': 0, 'TIME': 120.4601354598999, 'LOSSES': {'G': tensor(2.2561, device='cuda:0', grad_fn=<SubBackward0>), 'D': tensor(0., device='cuda:0', grad_fn=<AddBackward0>), 'Dfake': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'Dreal': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'OCR_fake': tensor(1.0751, device='cuda:0', grad_fn=<MulBackward0>), 'OCR_real': tensor(6.4343, device='cuda:0', grad_fn=<MeanBackward0>), 'w_fake': tensor(2.8839, device='cuda:0', grad_fn=<MulBackward0>), 'w_real': tensor(4.9050, device='cuda:0', grad_fn=<MeanBackward0>), 'cycle1': 0, 'cycle2': 0, 'lda1': 0, 'lda2': 0, 'KLD': 0}}


KeyboardInterrupt: 

COMET ERROR: Due to connectivity issues, there's an error in processing the heartbeat. The experiment's status updates might be inaccurate until the connection issues are resolved.
COMET ERROR: Due to an internal error, some network resiliency features were disabled which could lead to data loss. Contact us at support@comet.com. Error details: "register_messages: failed to insert messages into DB, reason: OperationalError('database or disk is full')"
COMET WARNING: Failed to log system metrics: [sys.ram,sys.cpu,sys.load]


: 

In [7]:
!pip install comet_ml

  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/729.6 kB ? eta -:--:--
   -------------- ------------------------- 262.1/729.6 kB ? eta -:--:--
   ---------------------------------------- 729.6/729.6 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/957.4 kB ? eta -:--:--
   --------------------------------------- 957.4/957.4 kB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/780.0 kB ? eta -:--:--
   --------------------------------------- 780.0/780.0 kB 32.4 MB/s eta 0:00:00
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)

   ----------------------------------------  0/13 [everett]
   ------ ---------------------------------  2/13 [wrapt]
   --------- ------------------------------  3/13 [simplejson]
   --------- ------------------------------  3/13 [simplejson]
   --------- ------------------------------  3/13 [simplejson]
   ------------ ---------------------------  4/13 [s

In [16]:
print(model)

TRGAN(
  (netG): Generator(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_